In [2]:
import pandas as pd

df_stations = pd.read_csv('bike_stations_vitoria.csv')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
import requests
import pandas as pd

foursquare_api_key = "KO0TPXQX4LNZZIOI1YHJEFBSBH3D4FP111CFS1JBKEJM142H"

headers = {
    "accept": "application/json",
    "Authorization": foursquare_api_key
}

def query_foursquare(lat, lon):
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        "ll": f"{lat},{lon}",
        "radius": 1000,
        "categories": "13065,13003",
        "limit": 20
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [4]:
poi_list = []

for station in df_stations.itertuples():
    print(f"Querying Foursquare for station: {station.name}")
    result = query_foursquare(station.latitude, station.longitude)
    print("Found results:", len(result.get('results', [])))
    
    for place in result.get('results', []):
        poi_list.append({
            "station_name": station.name,
            "poi_name": place["name"],
            "category": place["categories"][0]["name"] if place.get("categories") else None,
            "latitude": place["geocodes"]["main"]["latitude"],
            "longitude": place["geocodes"]["main"]["longitude"]
        })

print("Total POIs found:", len(poi_list))

Querying Foursquare for station: Mercado da Vila Rubim - Vitória
Found results: 0
Querying Foursquare for station: Santo Antônio - Prainha - Vitória
Found results: 0
Querying Foursquare for station: Fábrica de Idéias - Vitória
Found results: 0
Querying Foursquare for station: Reta da Penha - Vitória
Found results: 0
Querying Foursquare for station: SOE Jardim Camburi - Kids - Vitória
Found results: 0
Querying Foursquare for station: Praça dos Desejos - Vitória
Found results: 0
Querying Foursquare for station: SOE Beira Mar - Vitória
Found results: 0
Querying Foursquare for station: Baia de Skate - Praia de Camburi - Vitória
Found results: 0
Querying Foursquare for station: Prefeitura Municipal de Vitória - Vitória
Found results: 0
Querying Foursquare for station: SICOOB Central - Vitória
Found results: 0
Querying Foursquare for station: Goiabeiras - Vitória
Found results: 0
Querying Foursquare for station: Praça do Papa - Vitória
Found results: 0
Querying Foursquare for station: Itarar

Put your parsed results into a DataFrame

In [5]:
df_fsq = pd.DataFrame(poi_list)
pd.set_option('display.max_columns', None)
df_fsq.head(10)

""


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [11]:
import requests
import pandas as pd

yelp_api_key = "dFX5_FTn-DQiP5rq8uPf4ia0HnsAVUH2s3QVIIGbK2b9301GpZ6QgIUGavowJqRzxqXsPfSp1OU_xpuhISx1Pwt7qp-t3iOWOiUl8QVoliRx6I3sqN_Qw1FY_nptaHYx"
yelp_headers = {
    "Authorization": f"Bearer {yelp_api_key}"
}

def query_yelp(lat, lon):
    url = "https://api.yelp.com/v3/businesses/search"
    params = {
        "latitude": lat,
        "longitude": lon,
        "radius": 1000,
        "categories": "restaurants,bars",
        "limit": 20
    }
    response = requests.get(url, headers=yelp_headers, params=params)
    return response.json()

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [20]:
yelp_poi_list = []

for station in df_stations.itertuples():
    result = query_yelp(station.latitude, station.longitude)
    for business in result.get("businesses", []):
        yelp_poi_list.append({
            "station_name": station.name,
            "poi_name": business["name"],
            "category": business["categories"][0]["title"] if business.get("categories") else None,
            "rating": business.get("rating"),
            "latitude": business["coordinates"]["latitude"],
            "longitude": business["coordinates"]["longitude"]
        })

Put your parsed results into a DataFrame

In [21]:
import pandas as pd

df_yelp = pd.DataFrame(yelp_poi_list)
pd.set_option('display.max_columns', None)

df_yelp.sample(20)


,station_name,poi_name,category,rating,latitude,longitude
273,UFES Maruípe - Vitória,Self Service do Chico,Restaurants,0.0,-20.298780,-40.323799
84,SOE Jardim Camburi - Kids - Vitória,Padaria e Confeitaria Artesanal Delicias,Cafes,5.0,-20.257383,-40.267109
38,Santo Antônio - Prainha - Vitória,Restaurante Ki Delícia,Restaurants,0.0,-20.320749,-40.343590
518,Sagrada Família - Vitória,BAR V8,Bars,0.0,-20.257441,-40.268372
214,Goiabeiras - Vitória,Mister Frango - Fast Food & Beer,Fast Food,0.0,-20.275119,-40.292269
513,Sagrada Família - Vitória,Caiana Bar,Bars,5.0,-20.259443,-40.269132
308,Praça dos Namorados - Kids - Vitória,Sushimar Cozinha Japonesa restaurante,Restaurants,4.0,-20.294259,-40.293609
44,Fábrica de Idéias - Vitória,A Cantina Badaue,Restaurants,0.0,-20.317560,-40.324410
663,EDP - Vitória,Ponto 5 Restaurante,Restaurants,0.0,-20.311224,-40.302723
503,Sagrada Família - Vitória,Recanto Gaúcho,Steakhouses,4.0,-20.263547,-40.264405


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In my testing, Yelp provided more complete data. It consistently returned a higher number of POIs per station, included detailed attributes like user ratings, category and full business names. Foursquare, on the other hand, often returned fewer results or required more handling to get similar details. Therefore, based on both coverage and richness of attributes, Yelp had the edge.

Get the top 10 restaurants according to their rating

In [15]:
top10_yelp = df_yelp.sort_values(by="rating", ascending=False).head(10)
top10_yelp

,station_name,poi_name,category,rating,latitude,longitude
220,Praça do Papa - Vitória,Papaguth,Restaurants,5.0,-20.315954,-40.294182
584,Aquaviário - Vitória,Pienza Pizzaria,Pizza,5.0,-20.308531,-40.293980
141,Baia de Skate - Praia de Camburi - Vitória,Caiana Bar,Bars,5.0,-20.259443,-40.269132
354,SOE Jardim Camburi - Vitória,Padaria e Confeitaria Artesanal Delicias,Cafes,5.0,-20.257383,-40.267109
598,Gurigica - Vitória,Pirao,Breweries,5.0,-20.295601,-40.291668
591,Gurigica - Vitória,Restaurante Praia do Canto,Buffets,5.0,-20.295571,-40.290229
590,Gurigica - Vitória,Gulover,Brazilian,5.0,-20.301165,-40.295924
589,Gurigica - Vitória,Papaguth,Restaurants,5.0,-20.315954,-40.294182
588,Gurigica - Vitória,Don Camaleone,Pizza,5.0,-20.305040,-40.294972
477,Ponte de Camburi - Vitória,Don Camaleone,Pizza,5.0,-20.305040,-40.294972


In [22]:
df_yelp.to_csv("yelp_data.csv", index=False)